In [15]:
import numpy as np
import scipy.integrate as sci
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numba as nb

In [33]:

def smk(x, y, m, k):
    """
    Generate a subset of points from a 2D trajectory using a sliding window approach.
    
    Parameters:
    - x (array of float): The x-coordinates of the trajectory.
    - y (array of float): The y-coordinates of the trajectory.
    - m (int): The size of the sliding window.
    - k (int): The step size between successive windows.
    
    Returns:
    - ndarray: A 2D NumPy array containing points sampled according to the sliding window parameters.
    """
    N = len(x)
    num_points = int(np.floor((N - m) / k))
    s = np.empty((num_points, 2))  # Pre-allocate a NumPy array
    for i in range(num_points):
        index = m + (i * k) - 1
        s[i, 0] = x[index]
        s[i, 1] = y[index]
    return s


def lmk(x, y, m, k):
    """
    Calculate the approximate length of segments of a 2D trajectory using a sliding window.
    
    Parameters:
    - x (array of float): The x-coordinates.
    - y (array of float): The y-coordinates.
    - m (int): The size of the sliding window.
    - k (int): The step size between successive windows.
    
    Returns:
    - float: The scaled average distance covered in each segment of the trajectory.
    """
    points = smk(x, y, m, k)
    total_distance = 0.0
    for i in range(len(points) - 1):
        total_distance += np.linalg.norm(points[i + 1] - points[i])
    if len(points) > 1:
        return (total_distance * (len(x) - m)) / (k * k * (len(points)))
    else:
        return 0.0



def lk(x, y, k):
    """
    Compute the average length of a 2D trajectory using multiple sliding windows to enhance accuracy.
    
    Parameters:
    - x (array of float): The x-coordinates.
    - y (array of float): The y-coordinates.
    - k (int): The number of sliding windows to apply.
    
    Returns:
    - float: The average length computed over all specified window configurations.
    """
    moyenne = 0.0
    for m in range(1, k + 1):
        moyenne += lmk(x, y, m, k)
    return moyenne / k

def find_intercepts_2d(x, y, value):
    """
    Identify the intercepts where a 2D trajectory crosses a horizontal line at a specified value on the y-axis.
    
    Parameters:
    - x (array of float): The x-coordinates of the trajectory.
    - y (array of float): The y-coordinates of the trajectory.
    - value (float): The value on the y-axis to check for intercepts.
    
    Returns:
    - tuple of arrays: Arrays containing the x and y coordinates of the points where the trajectory crosses the specified value.
    """
    x_intercept = []
    y_intercept = []
    for i in range(len(y) - 1):
        if (y[i] < value and y[i + 1] > value) or (y[i] > value and y[i + 1] < value):
            # Linear interpolation to find the precise intercept point
            frac = (value - y[i]) / (y[i + 1] - y[i])
            x_intercept.append(x[i] + frac * (x[i + 1] - x[i]))
            y_intercept.append(value)
    return np.array(x_intercept), np.array(y_intercept)


def dimension_2D(x, y, ks=np.arange(1, 20)):
    """
    Apply the Lk method to compute the trajectory length for various configurations of sliding window sizes, 
    evaluated at the intercepts with a specified horizontal plane.
    
    Parameters:
    - x (array of float): The x-coordinates of the trajectory.
    - y (array of float): The y-coordinates of the trajectory.
    - z (array of float): The z-coordinates of the trajectory.
    - height (float): The height of the plane to find intercepts.
    - ks (array of int, optional): The range of k-values (window sizes) to use.
    
    Returns:
    - tuple: Two arrays, the first containing the k-values used, and the second containing the logarithm of the Lk lengths.
    """
    x_fit = []
    y_fit = []
    for i in range(len(ks)):
        y_fit.append(np.log(lk(x, y, ks[i])))
        x_fit.append(np.log(1 / ks[i]))
    return np.polyfit(x_fit, y_fit, 1)[0]